In [ ]:
#Importing required packages.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import numpy as np
from matplotlib import style
import seaborn as sns

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
%matplotlib inline

![](https://cdn-coiao.nitrocdn.com/CYHudqJZsSxQpAPzLkHFOkuzFKDpEHGF/assets/static/optimized/wp-content/uploads/2021/04/f85283410c7e35104310d6d678ed7890.supervised-learning-data_vs_unsupervised-learning-data.png)

# **Loading dataset**

In [ ]:
#Loading dataset
df = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
#Let's check how the data is distributed
df.head()

In [ ]:
df.shape

In [ ]:
df.hist(bins= 40, figsize=(14, 8), grid= False)
plt.tight_layout()

plt.show()

In [ ]:
def plotBoxplot(data):
    fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(10,8))
    fig.tight_layout(pad=4.0)

    col = 0
    row = 0
    colors = ['#bad9e9', '#7ab6d6', '#3c8abd']

    for i, column in enumerate(data.columns):
        sns.boxplot(y=column, data=data, ax=axes[row][col], color=colors[col])

        if (i + 1) % 3 == 0:
            row += 1
            col = 0
        else:
            col += 1
            
plotBoxplot(df[['chlorides', 'residual sugar', 'free sulfur dioxide','sulphates' , 'alcohol', 'total sulfur dioxide']])

![](https://www.learnbyexample.org/wp-content/uploads/r/typical-box-whisker-plot.png) 

In [ ]:
df.describe()

In [ ]:
df['quality'].value_counts(sort = False).plot(kind='bar', title = 'Target Variable - Quality',  rot=0)
plt.show()

**Correlation**

Correlation is an important statistic that tells us how two sets of values are related to each other. A positive correlation indicates that the values tend to increase with one another and a negative correlation indicates that values in one set tend to decrease with an increase in the other set. 

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True, cmap = 'coolwarm',linewidth=1.5,linecolor='Black')

# **Data Pre-processing**

1. Removing Duplicates
2. Missing values
3. Target Value Simplification

In [ ]:
#checking for duplicates
print('Number of Duplicate rows', df[df.duplicated(keep=False)].shape[0])

df[df.duplicated(keep=False)].head(6)

In [ ]:
##Remove Dulpicates
df.drop_duplicates(inplace=True)

In [ ]:
##Missing values
df.isna().sum()

In [ ]:
#Target Value Simplification
# dividing the target variable (quality) into 3 class.

# class1: bad (qulaity 3,4), Class2: Avergae (qulaity 5,6,7), Class3: Good (qulaity 8)
reviews = []
for i in df['quality']:
    if i >= 3 and i <= 4:
        reviews.append('1')
    elif i >= 5 and i <= 7:
        reviews.append('2')
    elif i == 8:
        reviews.append('3')
df['Reviews'] = reviews

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)

df['quality'].value_counts(sort = False).plot(kind='bar', title = 'Quality',  rot=0, ax=axes[0], figsize=(8, 4))
df['Reviews'].value_counts(sort = False).plot(kind='bar', title = 'Reviews',  rot=0, ax=axes[1])

plt.show()

# class1: bad (qulaity 3,4), Class2: Avergae (qulaity 5,6,7), Class3: Good (qulaity 8)

# **Feature Enginering**
1. Normalisation
2. Train-test split 
3. Balncing the data

In [ ]:
df.drop('quality', axis= 1, inplace=True)

In [ ]:
import numpy as np

X= df.iloc[:,:-1]  #input varibales 
y= df.iloc[:,-1]   #output variable

X.shape, y.shape

**Normalization**

Normalization is to change the values of numeric columns in the dataset to use a common scale, without distorting differences in the ranges of values or losing information

In [ ]:
#Normlisation

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_new = sc.fit_transform(X)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
fig.suptitle('Before and after Normlisation ')
ax1.plot(X, )
ax2.plot(X_new)
plt.plot()

**Train-test split**

Trainig 80 % , Testing 20 %

![](https://www.researchgate.net/profile/Brian-Mwandau/publication/325870973/figure/fig6/AS:639531594285060@1529487622235/Train-Test-Data-Split.png)

In [ ]:
#data spliting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=4)

In [ ]:
X_train.shape, X_test.shape

**Synthetic Minority Oversampling Technique** is an oversampling technique  

SMOTE works by utilizing a **k-nearest neighbor** algorithm to create synthetic data. SMOTE first start by choosing random data from the minority class, then k-nearest neighbors from the data are set. Synthetic data would then made between the random data and the randomly selected k-nearest neighbor. Let me show you the example below.

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR8mNYVxG5cs9ejhYWudfVbJt70ag70dThK4g&usqp=CAU)

In [ ]:
import collections
print("Before over sampling :", collections.Counter(y_train))

In [ ]:
sm = SMOTE(sampling_strategy='auto',
    random_state=1,
    k_neighbors=5,
    n_jobs=None,)

X_train, y_train = sm.fit_resample(X_train, y_train)

In [ ]:
print("change is value of count after using smote:", collections.Counter(Y_train))

# **Model**

In [ ]:
models = [LinearSVC(),  KNeighborsClassifier(), RandomForestClassifier(),
          DecisionTreeClassifier(), GradientBoostingClassifier(), GaussianNB()]

model_names = ['LinearSVM', 'KNearestNeighbors', 'RandomForestClassifier', 'DecisionTree',
               'GradientBoostingClassifier', 'GaussianNB']

acc = []

for model in range(len(models)):
    clf = models[model]
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    acc.append(accuracy_score(pred, y_test))

models = {'Modelling Algorithm': model_names, 'Accuracy': acc}

In [ ]:
models_df = pd.DataFrame(models)

In [ ]:
models_df

# Hyperparameter Tuning - RandomForestClassifier

![](https://www.freecodecamp.org/news/content/images/2020/08/how-random-forest-classifier-work.PNG)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 42)

from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 30, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
best_random = rf_random.best_estimator_

best_random.fit(X_train, y_train)

predictions = best_random.predict(X_test)

accuracy_score(predictions, y_test)

In [ ]:
(pd.Series(best_random.feature_importances_, index=X.columns).plot(kind='barh', title = 'Feature Importance'))    
plt.plot